In [3]:
!pip install transformers
!pip install pandas
!pip install scikit-learn
!pip install imbalanced-learn
!pip install emoji
!pip install google-play-scraper
!pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 7.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import re
import emoji
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Load data
df = pd.read_csv("line_reviews_labeled_with_confidence.csv")
print("Data shape:", df.shape)
print(df.head())

# Tampilkan value counts untuk setiap topik
print("\nDistribusi Topic 1 (Pengalaman Umum Penggunaan LINE):")
print(df['Topic_1_Pengalaman_Umum_Penggunaan_LINE'].value_counts())
print("\nDistribusi Topic 2 (Fitur Tambahan):")
print(df['Topic_2_Fitur_Tambahan'].value_counts())
print("\nDistribusi Topic 3 (Login dan Registrasi Akun):")
print(df['Topic_3_Login_dan_Registrasi_Akun'].value_counts())

def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    # 1. lowercase
    text = text.lower()
    # 2. hapus URL
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    # 3. hapus emoji
    all_chars = list(text)
    text = "".join([c for c in all_chars if not emoji.is_emoji(c)])
    # 4. hapus angka
    text = re.sub(r"\d+", "", text)
    # 5. hapus karakter non-alfabet
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # 6. hapus spasi ganda
    text = re.sub(r"\s+", " ", text).strip()
    return text

norm_dict = {
    "gak": "tidak", "ga": "tidak", "gk": "tidak", "tdk": "tidak", "enggak": "tidak",
    "nggak": "tidak", "kagak": "tidak",
    "gw": "saya", "gua": "saya", "aku": "saya", "sy": "saya", "gue": "saya",
    "lu": "kamu", "lo": "kamu", "km": "kamu", "kalian": "kamu",
    "yg": "yang", "kalo": "kalau", "klo": "kalau",
    "krn": "karena", "karna": "karena",
    "utk": "untuk", "untk": "untuk",
    "dgn": "dengan", "dlm": "dalam",
    "sdh": "sudah", "udh": "sudah", "udah": "sudah",
    "blm": "belum",
    "jg": "juga", "jga": "juga",
    "tp": "tetapi", "tapi": "tetapi",
    "aja": "saja", "aj": "saja",
    "bgt": "sangat", "banget": "sangat",
    "knp": "kenapa", "napa": "kenapa",
    "gmn": "bagaimana", "gimana": "bagaimana",
    "bs": "bisa", "bisaa": "bisa", "gabisa": "tidak bisa", "ga bisa": "tidak bisa",
    "trus": "terus", "trs": "terus",
    "jd": "jadi", "jdi": "jadi",
    "pdhl": "padahal",
    "bnyk": "banyak",
    "sm": "sama",
    "lbh": "lebih",
    "dr": "dari",
    "eror": "error", "erorr": "error", "errorr": "error",
    "apk": "aplikasi", "apps": "aplikasi", "aplikasine": "aplikasi",
    "hp": "ponsel", "handphone": "ponsel",
    "download": "unduh", "donlot": "unduh",
    "update": "pembaruan", "updet": "pembaruan",
    "login": "masuk", "log in": "masuk", "sign in": "masuk",
    "log out": "keluar",
    "notif": "notifikasi",
    "verif": "verifikasi",
    "no": "nomor", "nmr": "nomor",
    "pw": "kata sandi", "password": "kata sandi", "sandi": "kata sandi",
    "chat": "pesan", "chatting": "pesan",
    "call": "panggilan", "nelpon": "panggilan", "telfon": "panggilan",
    "voom": "timeline", "line voom": "timeline",
    "tolong": "mohon", "pls": "mohon", "please": "mohon", "plis": "mohon",
    "balikin": "kembalikan",
    "ilang": "hilang",
    "tau": "tahu",
    "liat": "lihat",
    "cuman": "hanya", "cuma": "hanya",
    "makasih": "terima kasih", "tq": "terima kasih",
    "mulu": "terus", "melulu": "terus",
    "bener": "benar",
    "sampe": "sampai",
    "kapan": "kapan",
    "kocakk": "kocak"
}

custom_stopwords = {
    "dan", "atau", "tetapi", "tapi", "juga",
    "yang", "di", "ke", "dari", "pada", "dalam", "untuk", "bagi", "kepada", "oleh",
    "ini", "itu", "tersebut", "disini", "disitu",
    "saya", "aku", "kamu", "dia", "mereka", "kita", "kami", "anda", "kalian",
    "bisa", "dapat", "akan", "sedang", "sudah", "telah", "masih", "belum",
    "ada", "adalah", "ialah", "merupakan", "sebagai", "seperti",
    "sih", "dong", "deh", "kok", "lah", "mah", "kan", "pun", "doang",
    "nya",
    "saja", "aja",
    "padahal", "walaupun", "meskipun",
    "karena", "sebab", "akibat", "sehingga", "maka",
    "terus", "lalu", "kemudian", "akhirnya",
    "sangat", "banget", "sekali", "terlalu",
    "mohon", "tolong", "harap", "silakan",
    "terima", "kasih", "makasih",
    "tanya", "tahu", "kasih", "banyak", "sedikit", "kurang", "lebih",
    "hari", "tanggal", "bulan", "tahun", "jam", "waktu",
    "kali", "tiap", "setiap",
    "apa", "kenapa", "bagaimana", "dimana", "siapa",
    "halo", "hai", "min", "admin", "kak", "gan", "sis", "bro"
}

factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Terapkan cleaning ke kolom 'content'
df['content_clean'] = df['content'].apply(clean_text)

def advanced_preprocessing(text: str) -> str:
    if not isinstance(text, str) or not text:
        return ""

    words = text.split()

    # Tahap A: Normalisasi Kata (Slang -> Baku)
    words = [norm_dict.get(w, w) for w in words]

    # Tahap B: Stopword Removal (Hapus kata umum yang tidak penting)
    # Kita lakukan sebelum stemming agar proses stemming lebih cepat
    words = [w for w in words if w not in custom_stopwords]

    # Tahap C: Stemming (Mengembalikan ke kata dasar)
    # Gabung dulu jadi string karena Sastrawi memproses kalimat
    text_processed = " ".join(words)
    text_stemmed = stemmer.stem(text_processed)

    return text_stemmed

# --- 4. TERAPKAN KE DATAFRAME ---
# Kita gunakan kolom 'content_clean' yang sudah Anda buat sebelumnya sebagai input
print("Sedang melakukan normalisasi, stopword removal, dan stemming... (Mohon tunggu sebentar)")
df['content_clean'] = df['content_clean'].apply(advanced_preprocessing)

print("Sebelum drop:", df.shape)
# Drop baris yang memiliki nilai null di kolom penting
df = df.dropna(subset=['content_clean', 'Topic_1_Pengalaman_Umum_Penggunaan_LINE',
                       'Topic_2_Fitur_Tambahan', 'Topic_3_Login_dan_Registrasi_Akun'])
print("Sesudah drop:", df.shape)

# Tampilkan daftar unik untuk setiap topik
print("\nUnique values Topic 1:")
print(repr(df['Topic_1_Pengalaman_Umum_Penggunaan_LINE'].unique()))
print("\nUnique values Topic 2:")
print(repr(df['Topic_2_Fitur_Tambahan'].unique()))
print("\nUnique values Topic 3:")
print(repr(df['Topic_3_Login_dan_Registrasi_Akun'].unique()))

# Label encoding untuk setiap topik
le_topic1 = LabelEncoder()
le_topic2 = LabelEncoder()
le_topic3 = LabelEncoder()

df['label_topic1'] = le_topic1.fit_transform(df['Topic_1_Pengalaman_Umum_Penggunaan_LINE'])
df['label_topic2'] = le_topic2.fit_transform(df['Topic_2_Fitur_Tambahan'])
df['label_topic3'] = le_topic3.fit_transform(df['Topic_3_Login_dan_Registrasi_Akun'])

# Tampilkan mapping untuk setiap topik
print("\nMapping Topic 1 (Pengalaman Umum Penggunaan LINE):")
print(dict(zip(le_topic1.classes_, le_topic1.transform(le_topic1.classes_))))
print("\nMapping Topic 2 (Fitur Tambahan):")
print(dict(zip(le_topic2.classes_, le_topic2.transform(le_topic2.classes_))))
print("\nMapping Topic 3 (Login dan Registrasi Akun):")
print(dict(zip(le_topic3.classes_, le_topic3.transform(le_topic3.classes_))))

Data shape: (5323, 7)
       userName                                            content  score  \
0      旋律melody                                                👍👍👍      5   
1  Fathur Rohim                             Bagus untuk komunikasi      5   
2     May Syifa  halo, maaf kenapa fitur tema di line saya ngga...      2   
3       A L E X  GABISA LOGIN LAGI WOY 😒 AKUN ADA , LOGIN PAKE ...      1   
4   Alfi Ismail  tolong lah line nya di perbagus lagi kaya wa ....      1   

                 at Topic_1_Pengalaman_Umum_Penggunaan_LINE  \
0   12/21/2025 8:02                                 Positif   
1   12/21/2025 2:54                                 Positif   
2   12/21/2025 1:10                                  Netral   
3  12/20/2025 17:36                                  Netral   
4  12/20/2025 16:49                                  Netral   

  Topic_2_Fitur_Tambahan Topic_3_Login_dan_Registrasi_Akun  
0                 Netral                            Netral  
1             

In [ ]:
from sklearn.model_selection import StratifiedKFold, train_test_split
import numpy as np
from transformers import AutoTokenizer
from torch.utils.data import Dataset
import torch
import pandas as pd

# 1. Ekstrak teks dan label untuk setiap topik
print("Total data:", len(df))
print("Distribusi Topic 1:", pd.Series(df['label_topic1']).value_counts().to_dict())
print("Distribusi Topic 2:", pd.Series(df['label_topic2']).value_counts().to_dict())
print("Distribusi Topic 3:", pd.Series(df['label_topic3']).value_counts().to_dict())

# 2. Split 7:1:2 per aspek
splits = {}
for i in [1, 2, 3]:
    texts = df['content_clean'].tolist()
    labels = df[f'label_topic{i}'].tolist()

    print(f"\n=== TOPIC {i} SPLIT ===")
    print(f"Topic {i} - Original distribution: {pd.Series(labels).value_counts().to_dict()}")

    # Cek apakah bisa stratify (minimal 2 sampel per kelas)
    label_counts = pd.Series(labels).value_counts()
    can_stratify_first = all(count >= 2 for count in label_counts.values)

    # 70% train / 30% temp
    if can_stratify_first:
        X_train, X_temp, y_train, y_temp = train_test_split(
            texts, labels,
            test_size=0.30,
            random_state=42,
            stratify=labels
        )
    else:
        print(f"Topic {i}: Cannot stratify first split (some classes < 2 samples)")
        X_train, X_temp, y_train, y_temp = train_test_split(
            texts, labels,
            test_size=0.30,
            random_state=42
        )

    # Cek apakah bisa stratify untuk split kedua
    temp_label_counts = pd.Series(y_temp).value_counts()
    can_stratify_second = all(count >= 2 for count in temp_label_counts.values)

    # dari temp (30%): 1/3 → val (10%), 2/3 → test (20%)
    if can_stratify_second:
        X_val, X_test, y_val, y_test = train_test_split(
            X_temp, y_temp,
            test_size=2/3,
            random_state=42,
            stratify=y_temp
        )
    else:
        print(f"Topic {i}: Cannot stratify second split (some classes < 2 samples in temp)")
        X_val, X_test, y_val, y_test = train_test_split(
            X_temp, y_temp,
            test_size=2/3,
            random_state=42
        )

    splits[f't{i}'] = {
        'X_train': X_train, 'y_train': y_train,
        'X_val': X_val, 'y_val': y_val,
        'X_test': X_test, 'y_test': y_test,
    }

    print(f"Topic {i} - Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")
    print(f"Topic {i} - Train labels: {pd.Series(y_train).value_counts().to_dict()}")
    print(f"Topic {i} - Val labels: {pd.Series(y_val).value_counts().to_dict()}")
    print(f"Topic {i} - Test labels: {pd.Series(y_test).value_counts().to_dict()}")

Total data: 5323
Distribusi Topic 1: {1: 2961, 0: 1554, 2: 808}
Distribusi Topic 2: {1: 4053, 0: 1251, 2: 19}
Distribusi Topic 3: {1: 3326, 0: 1990, 2: 7}

=== TOPIC 1 SPLIT ===
Topic 1 - Original distribution: {1: 2961, 0: 1554, 2: 808}
Topic 1 - Train: 3726, Val: 532, Test: 1065
Topic 1 - Train labels: {1: 2073, 0: 1088, 2: 565}
Topic 1 - Val labels: {1: 296, 0: 155, 2: 81}
Topic 1 - Test labels: {1: 592, 0: 311, 2: 162}

=== TOPIC 2 SPLIT ===
Topic 2 - Original distribution: {1: 4053, 0: 1251, 2: 19}
Topic 2 - Train: 3726, Val: 532, Test: 1065
Topic 2 - Train labels: {1: 2837, 0: 876, 2: 13}
Topic 2 - Val labels: {1: 405, 0: 125, 2: 2}
Topic 2 - Test labels: {1: 811, 0: 250, 2: 4}

=== TOPIC 3 SPLIT ===
Topic 3 - Original distribution: {1: 3326, 0: 1990, 2: 7}
Topic 3 - Train: 3726, Val: 532, Test: 1065
Topic 3 - Train labels: {1: 2328, 0: 1393, 2: 5}
Topic 3 - Val labels: {1: 332, 0: 199, 2: 1}
Topic 3 - Test labels: {1: 666, 0: 398, 2: 1}


In [ ]:
# 3. IndobertDataset definition
from transformers import BertTokenizer, AutoModel
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = AutoModel.from_pretrained("indobenchmark/indobert-base-p1")

class IndobertDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        enc = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': enc['input_ids'].flatten(),
            'attention_mask': enc['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 4. Instantiate dataset untuk tiap aspek
print("\n" + "="*50)
print("MEMBUAT DATASET UNTUK SETIAP TOPIK")
print("="*50)

# Topic 1 datasets
train_dataset_t1 = IndobertDataset(texts=splits['t1']['X_train'],
                                   labels=splits['t1']['y_train'],
                                   tokenizer=tokenizer)
val_dataset_t1 = IndobertDataset(texts=splits['t1']['X_val'],
                                 labels=splits['t1']['y_val'],
                                 tokenizer=tokenizer)
test_dataset_t1 = IndobertDataset(texts=splits['t1']['X_test'],
                                  labels=splits['t1']['y_test'],
                                  tokenizer=tokenizer)

# Topic 2 datasets
train_dataset_t2 = IndobertDataset(splits['t2']['X_train'], splits['t2']['y_train'], tokenizer)
val_dataset_t2 = IndobertDataset(splits['t2']['X_val'], splits['t2']['y_val'], tokenizer)
test_dataset_t2 = IndobertDataset(splits['t2']['X_test'], splits['t2']['y_test'], tokenizer)

# Topic 3 datasets
train_dataset_t3 = IndobertDataset(splits['t3']['X_train'], splits['t3']['y_train'], tokenizer)
val_dataset_t3 = IndobertDataset(splits['t3']['X_val'], splits['t3']['y_val'], tokenizer)
test_dataset_t3 = IndobertDataset(splits['t3']['X_test'], splits['t3']['y_test'], tokenizer)

print(f"Dataset Topic 1 - Train: {len(train_dataset_t1)}, Val: {len(val_dataset_t1)}, Test: {len(test_dataset_t1)}")
print(f"Dataset Topic 2 - Train: {len(train_dataset_t2)}, Val: {len(val_dataset_t2)}, Test: {len(test_dataset_t2)}")
print(f"Dataset Topic 3 - Train: {len(train_dataset_t3)}, Val: {len(val_dataset_t3)}, Test: {len(test_dataset_t3)}")

# 5. Ringkasan untuk referensi
datasets = {
    'topic1': {
        'train': train_dataset_t1,
        'val': val_dataset_t1,
        'test': test_dataset_t1
    },
    'topic2': {
        'train': train_dataset_t2,
        'val': val_dataset_t2,
        'test': test_dataset_t2
    },
    'topic3': {
        'train': train_dataset_t3,
        'val': val_dataset_t3,
        'test': test_dataset_t3
    }
}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]


MEMBUAT DATASET UNTUK SETIAP TOPIK
Dataset Topic 1 - Train: 3726, Val: 532, Test: 1065
Dataset Topic 2 - Train: 3726, Val: 532, Test: 1065
Dataset Topic 3 - Train: 3726, Val: 532, Test: 1065


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import torch
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import os


In [ ]:
# ===================================================================
# FUNGSI COMPUTE METRICS (TANPA ACCURACY)
# ===================================================================

def compute_metrics(p):
    """
    Fungsi untuk menghitung metrik: precision, recall, f1-score, roc-auc
    """
    preds = p.predictions
    labels = p.label_ids

    # Get predicted classes
    pred_classes = np.argmax(preds, axis=1)

    # Precision, Recall, F1-Score
    prec, rec, f1, _ = precision_recall_fscore_support(labels, pred_classes, average='weighted', zero_division=0)

    # Coherence calculation (simplified as consistency metric)
    # Coherence di sini dihitung sebagai konsistensi prediksi dengan confidence
    max_probs = np.max(preds, axis=1)
    coherence = np.mean(max_probs)  # Average confidence as coherence proxy

    return {
        'precision': prec,
        'recall': rec,
        'f1': f1,
        'coherence': coherence
    }

In [ ]:
# ===================================================================
# TRAINING TOPIC 1 - PENGALAMAN UMUM PENGGUNAAN LINE
# ===================================================================

print("="*70)
print("TRAINING MODEL TOPIC 1: PENGALAMAN UMUM PENGGUNAAN LINE")
print("="*70)

# Hitung jumlah label unik untuk Topic 1
labels_t1 = [train_dataset_t1[i]['labels'].item() for i in range(len(train_dataset_t1))]
num_labels_t1 = len(set(labels_t1))
print(f"Jumlah label Topic 1: {num_labels_t1}")

# Load model untuk Topic 1
model_topic1 = AutoModelForSequenceClassification.from_pretrained(
    "indobenchmark/indobert-base-p1", # Changed 'model' to the model identifier string
    num_labels=num_labels_t1,
    problem_type="single_label_classification"
)
model_topic1.to("cuda" if torch.cuda.is_available() else "cpu")

# Training arguments untuk Topic 1
training_args_topic1 = TrainingArguments(
    output_dir="/content/indobert_topic1_model",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    warmup_steps=100,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="/content/logs_topic1",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to=None,
    optim="adamw_torch" # Specify a non-fused optimizer
)

# Trainer untuk Topic 1
trainer_topic1 = Trainer(
    model=model_topic1,
    args=training_args_topic1,
    train_dataset=train_dataset_t1,
    eval_dataset=val_dataset_t1,
    compute_metrics=compute_metrics
)

# Training Topic 1
print("Memulai training Topic 1...")
trainer_topic1.train()

TRAINING MODEL TOPIC 1: PENGALAMAN UMUM PENGGUNAAN LINE
Jumlah label Topic 1: 3


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Memulai training Topic 1...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Coherence
1,No log,0.351039,0.883608,0.879699,0.879789,2.719948
2,No log,0.349259,0.880070,0.879699,0.876991,3.097714
3,0.373700,0.404929,0.878619,0.879699,0.878303,3.859500


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=699, training_loss=0.3114132969846712, metrics={'train_runtime': 173.0442, 'train_samples_per_second': 64.631, 'train_steps_per_second': 4.039, 'total_flos': 735270445840896.0, 'train_loss': 0.3114132969846712, 'epoch': 3.0})

In [ ]:
# Evaluasi Topic 1
print("\n=== EVALUASI TOPIC 1 ===")
# 1. Evaluate on training set
train_eval_t1 = trainer_topic1.evaluate(train_dataset_t1)
# 2. Evaluate on test set
test_pred_t1 = trainer_topic1.predict(test_dataset_t1)
test_eval_t1 = compute_metrics(test_pred_t1)

# 3. Hasil metrik Topic 1
df_metrics_t1 = pd.DataFrame([
    {
        'split': 'train',
        'precision': train_eval_t1['eval_precision'],
        'recall': train_eval_t1['eval_recall'],
        'f1': train_eval_t1['eval_f1'],
        'coherence': train_eval_t1['eval_coherence']
    },
    {
        'split': 'test',
        'precision': test_eval_t1['precision'],
        'recall': test_eval_t1['recall'],
        'f1': test_eval_t1['f1'],
        'coherence': test_eval_t1['coherence']
    }
])

print("Hasil Evaluasi Topic 1:")
print(df_metrics_t1.round(4))

# Simpan model Topic 1
OUTPUT_DIR_TOPIC1 = "/content/indobert_topic1_model_final"
model_topic1.save_pretrained(OUTPUT_DIR_TOPIC1, safe_serialization=False)
tokenizer.save_pretrained(OUTPUT_DIR_TOPIC1)
print(f"Model Topic 1 disimpan di: {OUTPUT_DIR_TOPIC1}")


=== EVALUASI TOPIC 1 ===


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Hasil Evaluasi Topic 1:
   split  precision  recall      f1  coherence
0  train     0.9134  0.9117  0.9120      2.793
1   test     0.8725  0.8704  0.8707      2.714
Model Topic 1 disimpan di: /content/indobert_topic1_model_final


In [ ]:
print("\n" + "="*70)
print("TRAINING MODEL TOPIC 2: FITUR TAMBAHAN")
print("="*70)

# Hitung jumlah label unik untuk Topic 2
labels_t2 = [train_dataset_t2[i]['labels'].item() for i in range(len(train_dataset_t2))]
num_labels_t2 = len(set(labels_t2))
print(f"Jumlah label Topic 2: {num_labels_t2}")

# Load model untuk Topic 2
model_topic2 = AutoModelForSequenceClassification.from_pretrained(
    "indobenchmark/indobert-base-p1", # Changed 'MODEL_NAME' to the model identifier string
    num_labels=num_labels_t2,
    problem_type="single_label_classification"
)
model_topic2.to("cuda" if torch.cuda.is_available() else "cpu")

# Training arguments untuk Topic 2
training_args_topic2 = TrainingArguments(
    output_dir="/content/indobert_topic2_model",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="/content/logs_topic2",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to=None,
    optim="adamw_torch" # Specify a non-fused optimizer
)

# Trainer untuk Topic 2
trainer_topic2 = Trainer(
    model=model_topic2,
    args=training_args_topic2,
    train_dataset=train_dataset_t2,
    eval_dataset=val_dataset_t2,
    compute_metrics=compute_metrics
)

# Training Topic 2
print("Memulai training Topic 2...")
trainer_topic2.train()

# Evaluasi Topic 2
print("\n=== EVALUASI TOPIC 2 ===")
# 1. Evaluate on training set
train_eval_t2 = trainer_topic2.evaluate(train_dataset_t2)
# 2. Evaluate on test set
test_pred_t2 = trainer_topic2.predict(test_dataset_t2)
test_eval_t2 = compute_metrics(test_pred_t2)

# 3. Hasil metrik Topic 2
df_metrics_t2 = pd.DataFrame([
    {
        'split': 'train',
        'precision': train_eval_t2['eval_precision'],
        'recall': train_eval_t2['eval_recall'],
        'f1': train_eval_t2['eval_f1'],
        'coherence': train_eval_t2['eval_coherence']
    },
    {
        'split': 'test',
        'precision': test_eval_t2['precision'],
        'recall': test_eval_t2['recall'],
        'f1': test_eval_t2['f1'],
        'coherence': test_eval_t2['coherence']
    }
])

print("Hasil Evaluasi Topic 2:")
print(df_metrics_t2.round(4))

# Simpan model Topic 2
OUTPUT_DIR_TOPIC2 = "/content/indobert_topic2_model_final"
model_topic2.save_pretrained(OUTPUT_DIR_TOPIC2, safe_serialization=False)
tokenizer.save_pretrained(OUTPUT_DIR_TOPIC2)
print(f"Model Topic 2 disimpan di: {OUTPUT_DIR_TOPIC2}")


TRAINING MODEL TOPIC 2: FITUR TAMBAHAN
Jumlah label Topic 2: 3


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Memulai training Topic 2...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Coherence
1,No log,0.110832,0.959089,0.962406,0.960716,3.416114
2,No log,0.122720,0.957736,0.960526,0.958994,3.551120
3,0.256000,0.112473,0.967144,0.969925,0.968342,4.692455


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



=== EVALUASI TOPIC 2 ===


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Hasil Evaluasi Topic 2:
   split  precision  recall      f1  coherence
0  train     0.9902  0.9901  0.9887     4.7900
1   test     0.9549  0.9568  0.9555     4.6793
Model Topic 2 disimpan di: /content/indobert_topic2_model_final


In [ ]:
  # ===================================================================
# TRAINING TOPIC 3 - LOGIN DAN REGISTRASI AKUN
# ===================================================================

print("\n" + "="*70)
print("TRAINING MODEL TOPIC 3: LOGIN DAN REGISTRASI AKUN")
print("="*70)

# Hitung jumlah label unik untuk Topic 3
labels_t3 = [train_dataset_t3[i]['labels'].item() for i in range(len(train_dataset_t3))]
num_labels_t3 = len(set(labels_t3))
print(f"Jumlah label Topic 3: {num_labels_t3}")

# Load model untuk Topic 3
model_topic3 = AutoModelForSequenceClassification.from_pretrained(
    "indobenchmark/indobert-base-p1", # Changed 'MODEL_NAME' to the model identifier string
    num_labels=num_labels_t3,
    problem_type="single_label_classification"
)
model_topic3.to("cuda" if torch.cuda.is_available() else "cpu")

# Training arguments untuk Topic 3
training_args_topic3 = TrainingArguments(
    output_dir="/content/indobert_topic3_model",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="/content/logs_topic3",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to=None,
    optim="adamw_torch" # Specify a non-fused optimizer
)

# Trainer untuk Topic 3
trainer_topic3 = Trainer(
    model=model_topic3,
    args=training_args_topic3,
    train_dataset=train_dataset_t3,
    eval_dataset=val_dataset_t3,
    compute_metrics=compute_metrics
)

# Training Topic 3
print("Memulai training Topic 3...")
trainer_topic3.train()

# Evaluasi Topic 3
print("\n=== EVALUASI TOPIC 3 ===")
# 1. Evaluate on training set
train_eval_t3 = trainer_topic3.evaluate(train_dataset_t3)
# 2. Evaluate on test set
test_pred_t3 = trainer_topic3.predict(test_dataset_t3)
test_eval_t3 = compute_metrics(test_pred_t3)

# 3. Hasil metrik Topic 3
df_metrics_t3 = pd.DataFrame([
    {
        'split': 'train',
        'precision': train_eval_t3['eval_precision'],
        'recall': train_eval_t3['eval_recall'],
        'f1': train_eval_t3['eval_f1'],
        'coherence': train_eval_t3['eval_coherence']
    },
    {
        'split': 'test',
        'precision': test_eval_t3['precision'],
        'recall': test_eval_t3['recall'],
        'f1': test_eval_t3['f1'],
        'coherence': test_eval_t3['coherence']
    }
])

print("Hasil Evaluasi Topic 3:")
print(df_metrics_t3.round(4))

# Simpan model Topic 3
OUTPUT_DIR_TOPIC3 = "/content/indobert_topic3_model_final"
model_topic3.save_pretrained(OUTPUT_DIR_TOPIC3, safe_serialization=False)
tokenizer.save_pretrained(OUTPUT_DIR_TOPIC3)
print(f"Model Topic 3 disimpan di: {OUTPUT_DIR_TOPIC3}")


TRAINING MODEL TOPIC 3: LOGIN DAN REGISTRASI AKUN
Jumlah label Topic 3: 3


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Memulai training Topic 3...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Coherence
1,No log,0.276462,0.915286,0.917293,0.916046,3.668486
2,No log,0.214965,0.931839,0.932331,0.931768,3.772961
3,0.266200,0.279854,0.935561,0.936090,0.935507,4.390857


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



=== EVALUASI TOPIC 3 ===


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Hasil Evaluasi Topic 3:
   split  precision  recall      f1  coherence
0  train     0.9866  0.9879  0.9873     4.5974
1   test     0.9507  0.9512  0.9508     4.5057
Model Topic 3 disimpan di: /content/indobert_topic3_model_final


In [ ]:
import re
import pickle
import os
from typing import List, Union

class TextPreprocessor:
    """
    Text Preprocessor yang bisa disimpan sebagai model untuk deployment
    Mengganti dependency emoji dengan regex pattern sederhana
    """

    def __init__(self):
        # Emoji patterns yang umum (tanpa dependency emoji library)
        self.emoji_patterns = [
            r'[\U0001F600-\U0001F64F]',  # emoticons
            r'[\U0001F300-\U0001F5FF]',  # symbols & pictographs
            r'[\U0001F680-\U0001F6FF]',  # transport & map symbols
            r'[\U0001F1E0-\U0001F1FF]',  # flags (iOS)
            r'[\U00002702-\U000027B0]',  # Dingbats
            r'[\U000024C2-\U0001F251]',  # Enclosed characters
            r'[\U0001F900-\U0001F9FF]',  # Supplemental Symbols and Pictographs
            r'[\U0001FA70-\U0001FAFF]',  # Symbols and Pictographs Extended-A
            r'[\U00002600-\U000026FF]',  # Miscellaneous Symbols
            r'[\U00002700-\U000027BF]',  # Dingbats
        ]
        self.compiled_emoji_pattern = re.compile('|'.join(self.emoji_patterns))

        # Pattern untuk preprocessing lainnya
        self.url_pattern = re.compile(r"http\S+|www\S+|https\S+")
        self.number_pattern = re.compile(r"\d+")
        self.non_alpha_pattern = re.compile(r'[^a-zA-Z0-9\s]')
        self.multiple_space_pattern = re.compile(r"\s+")

    def clean_text(self, text: Union[str, None]) -> str:
        """
        Membersihkan teks dengan langkah-langkah preprocessing
        """
        if not isinstance(text, str) or text is None:
            return ""

        # 1. lowercase
        text = text.lower()

        # 2. hapus URL
        text = self.url_pattern.sub("", text)

        # 3. hapus emoji (tanpa library emoji)
        text = self.compiled_emoji_pattern.sub("", text)

        # 4. hapus angka
        text = self.number_pattern.sub("", text)

        # 5. hapus karakter non-alfabet
        text = self.non_alpha_pattern.sub('', text)

        # 6. hapus spasi ganda
        text = self.multiple_space_pattern.sub(" ", text).strip()

        return text

    def clean_batch(self, texts: List[str]) -> List[str]:
        """
        Membersihkan batch teks sekaligus
        """
        return [self.clean_text(text) for text in texts]

    def save_model(self, filepath: str):
        """
        Simpan preprocessor sebagai pickle file
        """
        with open(filepath, 'wb') as f:
            pickle.dump(self, f)
        print(f"TextPreprocessor model disimpan di: {filepath}")

    @staticmethod
    def load_model(filepath: str):
        """
        Load preprocessor dari pickle file
        """
        with open(filepath, 'rb') as f:
            preprocessor = pickle.load(f)
        print(f"TextPreprocessor model dimuat dari: {filepath}")
        return preprocessor

    def __repr__(self):
        return "TextPreprocessor(emoji_free=True, url_free=True, number_free=True)"

# ===================================================================
# FUNGSI UNTUK MEMBUAT DAN MENYIMPAN MODEL PREPROCESSOR
# ===================================================================

def create_and_save_preprocessor(save_path: str = "/content/text_preprocessor.pkl"):
    """
    Membuat dan menyimpan TextPreprocessor model
    """
    print("Membuat TextPreprocessor model...")

    # Buat instance preprocessor
    preprocessor = TextPreprocessor()

    # Test dengan beberapa contoh
    test_texts = [
        "Halo! Ini adalah contoh teks 123 😀 dengan emoji dan angka",
        "Cek website ini: https://www.google.com untuk info lebih lanjut!!!",
        "WOW!!! Aplikasi ini BAGUS BANGET 👍👍👍 rating 5/5 ⭐⭐⭐⭐⭐",
        "gak bisa login nih 😭😭 error terus"
    ]

    print("\n=== TEST PREPROCESSING ===")
    for i, text in enumerate(test_texts, 1):
        cleaned = preprocessor.clean_text(text)
        print(f"Original {i}: {text}")
        print(f"Cleaned {i}:  {cleaned}")
        print()

    # Simpan model
    preprocessor.save_model(save_path)

    return preprocessor

# ===================================================================
# DEMO PENGGUNAAN UNTUK STREAMLIT
# ===================================================================

def demo_streamlit_usage():
    """
    Demo cara penggunaan di Streamlit
    """
    print("=== DEMO PENGGUNAAN DI STREAMLIT ===")

    # Simulasi load model di Streamlit
    preprocessor_path = "/content/text_preprocessor.pkl"

    if os.path.exists(preprocessor_path):
        # Load model
        preprocessor = TextPreprocessor.load_model(preprocessor_path)

        # Contoh input user
        user_input = "Aplikasi LINE bagus banget! 😍😍 tapi fitur call suka error 😤 rating 4/5 ⭐⭐⭐⭐"

        # Preprocessing
        cleaned_input = preprocessor.clean_text(user_input)

        print(f"Input user: {user_input}")
        print(f"Hasil preprocessing: {cleaned_input}")

        return preprocessor, cleaned_input
    else:
        print("Model preprocessor belum dibuat. Jalankan create_and_save_preprocessor() terlebih dahulu.")
        return None, None

create_and_save_preprocessor()
demo_streamlit_usage()

In [36]:
# ==============================
# CELL 1: PREPROCESSING MODULE
# ==============================

# 1. DEFINISI KAMUS MANUAL (KONTROL PENUH DI SINI)
# ==============================================================================

# Definisikan kamus normalisasi (slang -> formal)
custom_normalization_dict = {
    # Singkatan umum
    "yg": "yang", "gak": "tidak", "ga": "tidak", "bgt": "banget",
    "dgn": "dengan", "tdk": "tidak", "jgn": "jangan", "krn": "karena",
    "utk": "untuk", "sdh": "sudah", "aja": "saja", "kalo": "kalau",
    "kln": "kalian", "sy": "saya", "aq": "aku", "km": "kamu",
    "gimana": "bagaimana", "knp": "kenapa", "bisa": "dapat",

    # Typo atau slang spesifik aplikasi
    "donlot": "unduh", "download": "unduh", "apk": "aplikasi",
    "app": "aplikasi", "eror": "error", "lemot": "lambat",
    "lag": "lambat", "bug": "kesalahan", "fix": "perbaiki",
    "update": "pembaruan", "updet": "pembaruan", "chat": "pesan",
    "notif": "notifikasi", "login": "masuk", "log in": "masuk",
    "sign in": "masuk", "out": "keluar", "telfon": "telepon",
    "call": "telepon", "vc": "panggilan video", "futu": "foto",
    "gak": "tidak", "ga": "tidak", "gk": "tidak", "tdk": "tidak", "enggak": "tidak",
    "nggak": "tidak", "kagak": "tidak",
    "gw": "saya", "gua": "saya", "aku": "saya", "sy": "saya", "gue": "saya",
    "lu": "kamu", "lo": "kamu", "km": "kamu", "kalian": "kamu",
    "yg": "yang", "kalo": "kalau", "klo": "kalau",
    "krn": "karena", "karna": "karena",
    "utk": "untuk", "untk": "untuk",
    "dgn": "dengan", "dlm": "dalam",
    "sdh": "sudah", "udh": "sudah", "udah": "sudah",
    "blm": "belum",
    "jg": "juga", "jga": "juga",
    "tp": "tetapi", "tapi": "tetapi",
    "aja": "saja", "aj": "saja",
    "bgt": "sangat", "banget": "sangat",
    "knp": "kenapa", "napa": "kenapa",
    "gmn": "bagaimana", "gimana": "bagaimana",
    "bs": "bisa", "bisaa": "bisa", "gabisa": "tidak bisa", "ga bisa": "tidak bisa",
    "trus": "terus", "trs": "terus",
    "jd": "jadi", "jdi": "jadi",
    "pdhl": "padahal",
    "bnyk": "banyak",
    "sm": "sama",
    "lbh": "lebih",
    "dr": "dari",
    "eror": "error", "erorr": "error", "errorr": "error",
    "apk": "aplikasi", "apps": "aplikasi", "aplikasine": "aplikasi",
    "hp": "ponsel", "handphone": "ponsel",
    "download": "unduh", "donlot": "unduh",
    "update": "pembaruan", "updet": "pembaruan",
    "login": "masuk", "log in": "masuk", "sign in": "masuk",
    "log out": "keluar",
    "notif": "notifikasi",
    "verif": "verifikasi",
    "no": "nomor", "nmr": "nomor",
    "pw": "kata sandi", "password": "kata sandi", "sandi": "kata sandi",
    "chat": "pesan", "chatting": "pesan",
    "call": "panggilan", "nelpon": "panggilan", "telfon": "panggilan",
    "voom": "timeline", "line voom": "timeline",
    "tolong": "mohon", "pls": "mohon", "please": "mohon", "plis": "mohon",
    "balikin": "kembalikan",
    "ilang": "hilang",
    "tau": "tahu",
    "liat": "lihat",
    "cuman": "hanya", "cuma": "hanya",
    "makasih": "terima kasih", "tq": "terima kasih",
    "mulu": "terus", "melulu": "terus",
    "bener": "benar",
    "sampe": "sampai",
    "kapan": "kapan",
    "kocakk": "kocak",
    "skrg": "sekarang"
}

# Definisikan stopwords (kata yang akan dibuang)
custom_stopwords_list = {
    # Kata hubung umum
    "yang", "dan", "di", "ke", "dari", "ini", "itu", "untuk", "pada",
    "adalah", "sebagai", "dengan", "karena", "jika", "namun", "tetapi",
    "atau", "serta", "oleh", "saat", "dalam", "bisa", "sudah", "ada",
    "akan", "apakah", "bagaimana", "kenapa", "siapa", "dimana",

    # Kata ganti orang (opsional, tergantung kebutuhan topik)
    "saya", "aku", "kamu", "dia", "mereka", "kita", "kami", "anda",

    # Kata umum yang tidak memberi makna sentimen/topik di konteks ini
    "line", "aplikasi", "nya", "sih", "dong", "deh", "kok", "mah",
    "tuh", "ya", "yah", "oh", "ok", "oke", "yaitu", "yakni", "sekali",
    "saja", "lagi", "pun", "masih", "belum", "juga", "buat", "pake",
    "pakai", "bikin", "biar", "terus", "jadi", "kan", "lah", "kah",
    "the", "and", "please", "to", "for", "is", "of", "in",
    "dan", "atau", "tetapi", "tapi", "juga",
    "yang", "di", "ke", "dari", "pada", "dalam", "untuk", "bagi", "kepada", "oleh",
    "ini", "itu", "tersebut", "disini", "disitu",
    "saya", "aku", "kamu", "dia", "mereka", "kita", "kami", "anda", "kalian",
    "bisa", "dapat", "akan", "sedang", "sudah", "telah", "masih", "belum",
    "ada", "adalah", "ialah", "merupakan", "sebagai", "seperti",
    "sih", "dong", "deh", "kok", "lah", "mah", "kan", "pun", "doang",
    "nya",
    "saja", "aja",
    "padahal", "walaupun", "meskipun",
    "karena", "sebab", "akibat", "sehingga", "maka",
    "terus", "lalu", "kemudian", "akhirnya",
    "sangat", "banget", "sekali", "terlalu",
    "mohon", "tolong", "harap", "silakan",
    "terima", "kasih", "makasih",
    "tanya", "tahu", "kasih", "banyak", "sedikit", "kurang", "lebih",
    "hari", "tanggal", "bulan", "tahun", "jam", "waktu",
    "kali", "tiap", "setiap",
    "apa", "kenapa", "bagaimana", "dimana", "siapa",
    "halo", "hai", "min", "admin", "kak", "gan", "sis", "bro", "loh"
}

import re
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# ------------------------------
# LOAD DATASET
# ------------------------------
try:
    df = pd.read_csv("line_reviews_labeled_with_confidence.csv")
    print(f"Dataset loaded: {df.shape}")
except FileNotFoundError:
    print("File tidak ditemukan, pakai data dummy.")
    df = pd.DataFrame({
        "content": [
            "Aplikasi line yg sekarang lemot bgt, gabisa dibuka",
            "Suka banget sama fiturnya, tapi notif sering telat",
            "Kenapa abis update malah sering error",
            "Gak bisa login padahal password udah bener"
        ]
    })

df = df.dropna(subset=["content"])
df["content"] = df["content"].astype(str)

# ------------------------------
# PREPROCESSING FUNCTIONS
# ------------------------------
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def tokenize_text(text):
    return text.split()

def normalize_merged_words(tokens):
    result = []
    for token in tokens:
        if re.match(r'\bga[a-z]+', token):
            token = re.sub(r'\bga([a-z]+)', r'tidak \1', token)
            result.extend(token.split())
        else:
            result.append(token)
    return result

def normalize_tokens(tokens, custom_normalization_dict):
    return [custom_normalization_dict.get(t, t) for t in tokens]

def remove_stopwords(tokens, custom_stopwords_list):
    return [t for t in tokens if t not in custom_stopwords_list and len(t) > 1]

stemmer = StemmerFactory().create_stemmer()

def stem_tokens(tokens):
    return [stemmer.stem(t) for t in tokens]

def preprocess_text(text, custom_normalization_dict, custom_stopwords_list):
    tokens = tokenize_text(clean_text(text))
    tokens = normalize_merged_words(tokens)
    tokens = normalize_tokens(tokens, custom_normalization_dict)
    tokens = remove_stopwords(tokens, custom_stopwords_list)
    tokens = stem_tokens(tokens)
    return [t for t in tokens if len(t) > 2]

# ------------------------------
# DEBUG FUNCTION (STEP BY STEP)
# ------------------------------
def debug_preprocessing(text, normalization_dict, stopwords):
    print("=== TEKS ASLI ===")
    print(text)

    step1 = clean_text(text)
    print("\n[1] Cleaning:", step1)

    step2 = tokenize_text(step1)
    print("\n[2] Tokenisasi:", step2)

    step3 = normalize_merged_words(step2)
    print("\n[3] Normalisasi Regex:", step3)

    step4 = normalize_tokens(step3, normalization_dict)
    print("\n[4] Normalisasi Dictionary:", step4)

    step5 = remove_stopwords(step4, stopwords)
    print("\n[5] Stopword Removal:", step5)

    step6 = stem_tokens(step5)
    print("\n[6] Stemming:", step6)

    final = [t for t in step6 if len(t) > 2]
    print("\n[7] Final Tokens:", final)

    return final

# ------------------------------
# APPLY PREPROCESSING
# ------------------------------
df["tokens"] = df["content"].apply(
    lambda x: preprocess_text(x, custom_normalization_dict, custom_stopwords_list)
)

df = df[df["tokens"].apply(len) > 0]
df["preprocessed_text"] = df["tokens"].apply(lambda x: " ".join(x))

print(f"Dokumen setelah preprocessing: {df.shape[0]}")

Dataset loaded: (5323, 7)
Dokumen setelah preprocessing: 5169


In [37]:
# Sampel teks panjang (Ulasan keluhan teknis)
long_sample_text = """
Balikin fitur line voom, padahal berguna banget loh
"""

# Jalankan debug
debug_preprocessing(
    long_sample_text,
    custom_normalization_dict,
    custom_stopwords_list
)

=== TEKS ASLI ===

Balikin fitur line voom, padahal berguna banget loh


[1] Cleaning: balikin fitur line voom padahal berguna banget loh

[2] Tokenisasi: ['balikin', 'fitur', 'line', 'voom', 'padahal', 'berguna', 'banget', 'loh']

[3] Normalisasi Regex: ['balikin', 'fitur', 'line', 'voom', 'padahal', 'berguna', 'banget', 'loh']

[4] Normalisasi Dictionary: ['kembalikan', 'fitur', 'line', 'timeline', 'padahal', 'berguna', 'sangat', 'loh']

[5] Stopword Removal: ['kembalikan', 'fitur', 'timeline', 'berguna']

[6] Stemming: ['kembali', 'fitur', 'timeline', 'guna']

[7] Final Tokens: ['kembali', 'fitur', 'timeline', 'guna']


['kembali', 'fitur', 'timeline', 'guna']

In [38]:
# ==============================
# CELL 2: TF-IDF MODULE
# ==============================

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

def build_tfidf(corpus_text, min_df=5, max_df=0.7):
    vectorizer = TfidfVectorizer(
        min_df=min_df,
        max_df=max_df,
        sublinear_tf=True,
        use_idf=True,
        norm=None
    )
    matrix = vectorizer.fit_transform(corpus_text)
    return vectorizer, matrix

def show_tfidf_sample(vectorizer, matrix, doc_index, top_n=15):
    feature_names = vectorizer.get_feature_names_out()
    values = matrix[doc_index].toarray().flatten()

    df_tfidf = pd.DataFrame({
        "term": feature_names,
        "tfidf": values
    })

    df_tfidf = df_tfidf[df_tfidf.tfidf > 0] \
                .sort_values("tfidf", ascending=False)

    print(f"\n=== TF-IDF TOP {top_n} | DOKUMEN {doc_index} ===")
    display(df_tfidf.head(top_n))

# BUILD TF-IDF
tfidf_vectorizer, tfidf_matrix = build_tfidf(df["preprocessed_text"])
print("TF-IDF shape:", tfidf_matrix.shape)


TF-IDF shape: (5169, 910)


In [39]:
# ==============================
# CELL: INSPEKSI SPESIFIK (INDEX 3574)
# ==============================

# Kita kunci indexnya ke 3574 (sesuai temuan di screenshot)
target_index = 3574

print(f"--- FOKUS INSPEKSI DOKUMEN KE-{target_index} ---")

# 1. Cek Teks Asli & Preprocessed
print(f"\n[A] Content Asli:")
print(f"    {df['content'].iloc[target_index]}")

print(f"\n[B] Preprocessed (Input TF-IDF):")
print(f"    {df['preprocessed_text'].iloc[target_index]}")

# 2. Cek Bobot TF-IDF
print(f"\n[C] Tabel Skor TF-IDF:")
show_tfidf_sample(tfidf_vectorizer, tfidf_matrix, doc_index=target_index, top_n=10)

--- FOKUS INSPEKSI DOKUMEN KE-3574 ---

[A] Content Asli:
    Balikin fitur line voom, padahal berguna banget loh

[B] Preprocessed (Input TF-IDF):
    kembali fitur timeline guna

[C] Tabel Skor TF-IDF:

=== TF-IDF TOP 10 | DOKUMEN 3574 ===


,term,tfidf
279,guna,3.788577
378,kembali,3.536913
243,fitur,3.370611
829,timeline,3.220907
